In [9]:
import numpy as np
import pandas as pd
from random import sample
from astropy.coordinates import SkyCoord
import astropy.units as u
import lsdb
from dask.distributed import Client
from catalog_filtering import bandFilterLenient, contains_PM
from pathlib import Path
print("Imported libraries")

Imported libraries


In [3]:
client = Client(n_workers=4)
print("Client Initialized")

Client Initialized


In [13]:
bandList = ['G','R','I','Z','Y']
class_star = None
spread_model = 0.05
magnitude_error = 0.05
check_flags = True
check_invalid_mags = True
query_string = bandFilterLenient(bandList,classStar=class_star,spreadModel=spread_model,magError=magnitude_error,flag=check_flags,invalidMags=check_invalid_mags)
des_cols = (
    [f'CLASS_STAR_{band}' for band in bandList] + 
    [f'FLAGS_{band}' for band in bandList] + 
    ['RA','DEC','COADD_OBJECT_ID'] + 
    [f'SPREAD_MODEL_{band}' for band in bandList] + 
    [f'WAVG_MAG_PSF_{band}' for band in bandList] + 
    [f'WAVG_MAGERR_PSF_{band}' for band in bandList]
)
k = 1
collinear_error_cutoff = 0.2
pm_speed_min = 2000 #units are milliseconds per year
pm_speed_max = 10**5
cone_search_rad = 50
max_neighbor_dist = 18
milliarc_degree_conversion = 1/(1000*3600)
# Parameters
n_samples = 100
search_radius_arcsec = 18
des_columns = ['RA', 'DEC']  # add more if needed
CATALOG_DIR = Path("../../catalogs")
DES_FILTERED_NAME = "des_light_filtered"
DES_FILTERED_DIR = CATALOG_DIR / DES_FILTERED_NAME
print("Defined local vars.")

Defined local vars.


In [15]:
des_catalog = (
    lsdb.read_hats(DES_FILTERED_DIR)
    .query(query_string)  # <- insert your filter
)
print("Defined catalog.")

Defined catalog.


In [19]:
partition_ids = des_catalog.get_partition_ids()



AttributeError: 'Catalog' object has no attribute 'get_partition_ids'

In [ ]:
# Function to count neighbors within search_radius_arcsec
def count_neighbors(row):
    ra, dec = row['RA'], row['DEC']
    nearby = des_catalog.cone_search(ra=ra, dec=dec, radius_arcsec=search_radius_arcsec)
    return len(nearby.to_pandas()) - 1  # exclude the object itself